In [6]:
import numpy as np
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel
from IPython.display import clear_output
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score


In [7]:
#On récupère les données des fichiers du dossier sentiment_labelled_sentences dans un dataframe
files = ["amazon_cells_labelled","imdb_labelled","yelp_labelled"]
liste_df = []
for f in files:
    liste_df.append(pd.read_csv("sentiment_labelled_sentences/"+f+".txt",sep='\t',names=["reviews","target"]))
df = pd.concat(liste_df, axis=0, ignore_index=True)
df

,reviews,target
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2743,I think food should have flavor and texture an...,0
2744,Appetite instantly gone.,0
2745,Overall I was not impressed and would not go b...,0
2746,"The whole experience was underwhelming, and I ...",0


In [8]:
#On récupère les target dans y et les reviews que l'on va passer dans le tokenizer
y = df['target']
reviews = df['reviews']

In [9]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
#Pour chaque review, on récupère l'output du tokenizer
X = []
i=0
for r in reviews:
    if i%50==49:
        clear_output(wait=True)
        print(str(i+1)+"/"+str(len(reviews)))
    with torch.no_grad():
        encoded_input = tokenizer(r, return_tensors='pt')
        output = model(**encoded_input)
    #On récupère l'output sans les couches cachées
    X.append(output[0][0][:,0].numpy())
    i+=1
X = np.array(X)

1000/2748


RuntimeError: The size of tensor a (1103) must match the size of tensor b (512) at non-singleton dimension 1

In [14]:
#On découpe les valeurs pour la classification
X_train, X_test, y_train, y_test = train_test_split(X,y)

ValueError: Found input variables with inconsistent numbers of samples: [1019, 2748]

In [15]:
#On entraine la régression logistique
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print("Precision : "+str(precision_score(y_test, y_pred,average="weighted")))
print("Rappel : "+str(recall_score(y_test, y_pred,average="weighted")))
print("Score F1 : "+str(f1_score(y_test, y_pred,average="weighted")))

Precision : 0.2595509450832576
Rappel : 0.5094614264919942
Score F1 : 0.3438987449801311


D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
